## Thermal Properties
#### Environment Setup

In [1]:
TChem_install_directory ='where/tchem/is/installed'

In [2]:
import sys
sys.path.append(TChem_install_directory+'/lib')
import pytchem
import numpy as np
import matplotlib.pyplot as plt
import time

In [3]:
sys.path.append(TChem_install_directory+'/example/runs/scripts/')
import pmixSample

## Pre-simulation
* Set variables; temperature and stoichiometric ratio (fuel/air).
* Convert from stoichiometric ratio to mass fraction of CH4, O2, N2 and AR.
* Create samples spaning over the variable ranges.

In [4]:
# Pressure, Temperature, phi(stoichiometric ratio)
one_atm = 101325 # [Pa]
TempMax = 1300   # Maximum temperature [K]
TempMin = 1000   # Minimum temperature [K]

PressureMax = 1*one_atm; # [Pa]
PressureMin = 1*one_atm; # [Pa]

phiMax = 3.0; # Maximum equivalence ratio [-]
phiMin = 0.8; # Minimum equivalence ratio [-]

In [5]:
Npp = 1
Npt = 2 
Npphi = 2
N = Npp*Npt*Npphi

pressure    = [PressureMin] #
temperature = np.linspace(TempMin, TempMax, Npt)
eqratio     = np.linspace(phiMin, phiMax, Npphi)
p, temp, phi = np.meshgrid(pressure, temperature,eqratio)
p    = p.reshape(np.size(p))       #flatten()
temp = temp.reshape(np.size(temp)) #flatten()
phi  = phi.reshape(np.size(phi))   #flatten()

In [6]:
Nvar = 6
sample = np.zeros([N,Nvar])
fuel ="CH4"
nC=1

In [7]:
for i in range(N):
    sample[i,0] = temp[i]
    sample[i,1] = p[i]
    Yp_fuel, Yr_o2, Yr_n2, Yr_ar = pmixSample.getMassFraction(nC,phi[i])
    sample[i,2] = Yp_fuel
    sample[i,3] = Yr_o2
    sample[i,4] = Yr_n2 
    sample[i,5] = Yr_ar

## TChem Simulation

### Initialize TChem Driver Object

* Initialization of Kokkos.
* Create a TChem driver object. 

In [8]:
pytchem.initialize()

In [9]:
tchem = pytchem.TChemDriver()

* Get help from TChem driver object.

In [10]:
help(tchem)

Help on TChemDriver in module pytchem object:

class TChemDriver(pybind11_builtins.pybind11_object)
 |  A class to manage data movement between numpy to kokkos views in TChem::Driver object
 |  
 |  Method resolution order:
 |      TChemDriver
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(...)
 |      __init__(self: pytchem.TChemDriver) -> None
 |  
 |  cloneGasKineticModel(...)
 |      cloneGasKineticModel(self: pytchem.TChemDriver) -> None
 |      
 |      Internally create clones of the kinetic model
 |  
 |  computeGasEnthapyMass(...)
 |      computeGasEnthapyMass(self: pytchem.TChemDriver) -> None
 |      
 |      Compute enthalpy mass and mixture enthalpy
 |  
 |  computeGasNetProductionRatePerMass(...)
 |      computeGasNetProductionRatePerMass(self: pytchem.TChemDriver) -> None
 |      
 |      Compute net production rate
 |  
 |  computeGasReactionRateConstants(...)
 |      computeGasReactionRateConstants(sel

### Create Kinetic Model 

* Inputs are the reactions mechanism files; in this case, we use the GRI3.0 gas reaction mechanism

In [11]:
inputs_directory = TChem_install_directory + '/example/data/ignition-zero-d/gri3.0/'
tchem.createGasKineticModel(inputs_directory+'chem.inp',inputs_directory+'therm.dat')

* Set number of samples
* Internally construct const object of the kinetic model and load them to device

In [12]:
tchem.setNumberOfSamples(N)
tchem.createGasKineticModelConstData()

### Set State Vector

* Get index for variables. 
* Pass a numpy array to the TChem object to set the state vector. 

In [13]:
Variables = ['T','P','CH4','O2','N2','AR']
indx=[]
for var in Variables:
    indx += [tchem.getStateVariableIndex(var)]

state = np.zeros([N, tchem.getLengthOfStateVector()])
for sp in range(N):
    state[sp,indx] = sample[sp,:]

In [14]:
tchem.setNumberOfSamples(N)
tchem.createStateVector()
tchem.setStateVector(state)

* compute mixture enthalpy and enthalpy per species both based mass 

$$ h=\sum_{k=1}^{k=N_{species} } Y_k h_k $$

In [15]:
tchem.computeGasEnthapyMass()

* get enthalpy per species $h_k$, where $k=1, cdots, N_{species}$

In [16]:
enthalpy_max_per_species = tchem.getGasEnthapyMass()

In [17]:
print('number of samples and number of species', np.shape(enthalpy_max_per_species))
print(enthalpy_max_per_species)

number of samples and number of species (4, 53)
[[ 1.02614945e+07  2.30747157e+08  1.65026304e+07  7.09615252e+05
   3.54350504e+06 -1.19799209e+07  1.27567256e+06 -2.88176584e+06
   6.08831026e+07  4.75115142e+07  2.99676752e+07  3.25849243e+07
   1.20694107e+07 -2.24075410e+06 -3.17164451e+06 -8.18249279e+06
   2.45579922e+06 -2.45764204e+06  9.93023744e+05  1.88697895e+06
  -4.75837837e+06  2.39581967e+07  1.03404817e+07  1.27490842e+07
   3.67640350e+06  6.08182937e+06 -6.46523620e+05  5.37711829e+06
   5.68010530e+04  3.13211139e+06  3.47881095e+07  2.51662176e+07
   1.36640637e+07 -7.79139716e+05  9.61364095e+06  3.78249123e+06
   1.45462251e+06  2.63124373e+06  4.35092857e+06  1.76997949e+07
   5.98929181e+06  1.01555921e+07  1.23359354e+07  4.98199239e+06
   6.69109162e+05 -1.76975622e+06  3.96766107e+06  7.66414981e+05
   3.65194263e+05  4.33097531e+06 -2.58074564e+05  1.87633682e+06
  -2.37306852e+06]
 [ 1.02614945e+07  2.30747157e+08  1.65026304e+07  7.09615252e+05
   3.5435

* get mixture enthalpy $h$

In [18]:
enthalpy_mix_max = tchem.getGasEnthapyMixMass()

In [19]:
print(enthalpy_mix_max)

[ 615332.07284743  304816.3460359  1015456.97961876  822990.66239356]


Finalize Kokkos. This deletes the TChem object and the data stored as Kokkos views

In [20]:
del(tchem)

In [21]:
pytchem.finalize()